<a href="https://colab.research.google.com/github/shubham-ai/-Sentiment-Analysis-with-an-RNN/blob/master/PytorchSentimentalAnalysisUsingRNN%26LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import os

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
%cd '/content/drive/My Drive/Sentimental analysis'


/content/drive/My Drive/Sentimental analysis


In [24]:
ls


 data/  'PytorchSentimentalAnalysisUsingRNN&LSTM.ipynb'


In [25]:
import numpy as np

with open('data/reviews.txt','r') as f:
    reviews = f.read()
with open('data/labels.txt', 'r') as f:
    labels = f.read()
    
print(reviews[:1000])
print("")
print(labels[:20])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

In [26]:
from string import punctuation

reviews = reviews.lower()
all_text = ''.join([c for c in reviews if c not in punctuation])
print(all_text[0:200])

bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s 


In [0]:
# split by new lince and spaces

reviews_split = all_text.split('\n')
all_text = ' '.join(reviews_split)
# print(reviews_split[0][0:1000])
words = all_text.split()

In [28]:
words[:40]
len(reviews_split)

25001

In [29]:
# feel free to use this import 
from collections import Counter

## Build a dictionary that maps words to integers
counts = Counter(words)
# counts.get gets the most uncommen value fist then other value
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = []
for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])
print(reviews_ints[0][0:10]) 
print(vocab_to_int["bromwell"])    





[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1]
21025


In [30]:
print("total numer of unique words", len((vocab_to_int)))



total numer of unique words 74072


In [31]:
#Encoding labels
print(labels[:9])
labels_split = labels.split('\n')
print(labels_split[0])

encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels_split])
len(encoded_labels),len(reviews_ints)

positive

positive


(25001, 25001)

In [32]:
review_lens = Counter([len(x) for x in reviews_ints])

review_lens[0],max(review_lens)

(1, 2514)

In [33]:
import numpy as np
non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review)!=0]


reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print('Number of reviews after removing outliers: ', len(reviews_ints))

Number of reviews after removing outliers:  25000


In [0]:
def pad_features(reviews_ints, seq_length):
    features = np.zeros((len(reviews_ints),seq_length),dtype=int)
    print("features.sahpe : ",features.shape)
    for i,row in enumerate(reviews_ints):
        #i=index, len(row) , row = partucular row index, seq is min_seq length from  
        features[i, -len(row): ]=np.array(row)[:seq_length]
    return features

In [0]:
# # # e.g for understanding of above e.g    
# # # a=np.array([1,2,4,5,3,3])
# a=np.zeros((1,10))

# b=np.ones((2,5))

# a[0,-5:]=b[1][:5]

# a[:,-10:],b

In [36]:
# test  implementation 

seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)


assert len(features)==len(reviews_ints)
assert len(features[0])==seq_length

print(len(features)*.8)

features.sahpe :  (25000, 200)
20000.0


In [37]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*0.8)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader

train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)


In [39]:
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[    0,     0,     0,  ...,    31,     1,    95],
        [    0,     0,     0,  ...,   527,    44,     8],
        [    0,     0,     0,  ...,   359,    44,     8],
        ...,
        [    0,     0,     0,  ...,  2436,  7466,  3441],
        [ 3604, 37834,     6,  ...,  9402,    68,     1],
        [  149,    10,   115,  ...,    41,   144,     5]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
        0, 0])


In [40]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [0]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        # So the contiguous function doesn't affect your target tensor at all, it just makes sure that it is stored in a contiguous chunk of memory.
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

In [0]:
# Hyperparams

vocab_size = len(vocab_to_int)+1
output_size = 1
embedding_dim = 400
hidden_dim = 512
n_layers = 5

net= SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)




In [44]:
print(net)


SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 512, num_layers=5, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [0]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)



In [46]:
# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/4... Step: 100... Loss: 0.610876... Val Loss: 0.676654
Epoch: 1/4... Step: 200... Loss: 0.697968... Val Loss: 0.693228
Epoch: 1/4... Step: 300... Loss: 0.693080... Val Loss: 0.695077
Epoch: 1/4... Step: 400... Loss: 0.685560... Val Loss: 0.695497
Epoch: 2/4... Step: 500... Loss: 0.697052... Val Loss: 0.693686
Epoch: 2/4... Step: 600... Loss: 0.693344... Val Loss: 0.692664
Epoch: 2/4... Step: 700... Loss: 0.693261... Val Loss: 0.693816
Epoch: 2/4... Step: 800... Loss: 0.693864... Val Loss: 0.692758
Epoch: 3/4... Step: 900... Loss: 0.697099... Val Loss: 0.694952
Epoch: 3/4... Step: 1000... Loss: 0.691570... Val Loss: 0.694092
Epoch: 3/4... Step: 1100... Loss: 0.693566... Val Loss: 0.693813
Epoch: 3/4... Step: 1200... Loss: 0.692929... Val Loss: 0.694327
Epoch: 4/4... Step: 1300... Loss: 0.693597... Val Loss: 0.694063
Epoch: 4/4... Step: 1400... Loss: 0.693987... Val Loss: 0.691352
Epoch: 4/4... Step: 1500... Loss: 0.693720... Val Loss: 0.695788
Epoch: 4/4... Step: 1600... Loss: 

In [47]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.694
Test accuracy: 0.500


In [0]:
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'



In [49]:
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])

    return test_ints

# test code and generate tokenized review
test_ints = tokenize_review(test_review_neg)
print(test_ints)

[[1, 247, 18, 10, 28, 108, 113, 14, 388, 2, 10, 181, 60, 273, 144, 11, 18, 68, 76, 113, 2, 1, 410, 14, 539]]


In [50]:
seq_length=200
features = pad_features(test_ints, seq_length)




features.sahpe :  (1, 200)


In [0]:
def predict(net, test_review, sequence_length=200):
    
    net.eval()
    
    # tokenize review
    test_ints = tokenize_review(test_review)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    # print custom response
    if(pred.item()==1):
        print("Positive review detected!")
    else:
        print("Negative review detected.")
        

In [0]:
test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'



In [53]:
 # call function
seq_length=200 # good to use the length that was trained on

predict(net, test_review_pos, seq_length)

features.sahpe :  (1, 200)
Prediction value, pre-rounding: 0.873216
Positive review detected!
